# NLTK Chapter 10

## Analyzing the Meaning of Sentences

*The html version of this chapter in the book is available [here](https://www.nltk.org/book/ch10.html "ch10").*



In [2]:
import nltk
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [3]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in China'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
q = ' '.join(answer)
print(q)

from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows: print(r[0], end=" ")



SELECT City FROM city_table WHERE   Country="china"
canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

In [4]:
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


In [5]:
lp = nltk.sem.logic.LogicParser()
print(lp.parse('-(P & Q)'))
print(lp.parse('P & Q'))
print(lp.parse('P | (R -> Q)'))
print(lp.parse('P <-> -- P'))


-(P & Q)
(P & Q)
(P | (R -> Q))
(P <-> --P)


In [6]:
lp = nltk.sem.logic.LogicParser()
SnF = lp.parse('SnF')
NotFnS = lp.parse('-FnS')
R = lp.parse('SnF -> -FnS')
prover = nltk.Prover9()
# prover.prove(NotFnS, [SnF, R])
# need to downloand prover9

In [7]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])
print(val['P'])
dom = set([])
g = nltk.Assignment(dom)
m = nltk.Model(dom, val)

print(m.evaluate('(P & Q)', g))
print(m.evaluate('-(P & Q)', g))
print(m.evaluate('(P & R)', g))
print(m.evaluate('(P | R)', g))


True
True
False
False
True


In [8]:
tlp = nltk.sem.logic.LogicParser(type_check=True)
parsed = tlp.parse('walk(angus)')
print(parsed.argument)
print(parsed.argument.type)
print(parsed.function)
print(parsed.function.type)




angus
e
walk
<e,?>


In [9]:
sig = {'walk': '<e, t>'}
parsed = tlp.parse('walk(angus)', sig)
parsed.type

t

In [11]:
lp = nltk.sem.logic.LogicParser()
print(lp.parse('dog(cyril)').free())
print(lp.parse('dog(x)').free())
print(lp.parse('own(angus, cyril)').free())
print(lp.parse('exists x.dog(x)').free())
print(lp.parse('((some x. walk(x)) -> sing(x))').free())
print(lp.parse('exists x.own(y, x)').free())




set()
{Variable('x')}
set()
set()
{Variable('x')}
{Variable('y')}


In [13]:
NotFnS = lp.parse('-north_of(f, s)')
SnF = lp.parse('north_of(s, f)')
R = lp.parse('all x. all y. (north_of(x, y) -> -north_of(y, x))')
prover = nltk.Prover9()
# prover.prove(NotFnS, [SnF, R])
# need to downloand prover9

FnS = lp.parse('north_of(f, s)')
# prover.prove(FnS, [SnF, R])

In [16]:
dom = {'b', 'o', 'c'}
v = """
    bertie => b
    olive => o
    cyril => c
    boy => {b}
    girl => {o}
    dog => {c}
    walk => {o, c}
    see => {(b, o), (c, b), (o, c)}
    """
val = nltk.Valuation.fromstring(v)
print(val)

print(('o', 'c') in val['see'])
print(('b', 'o') in val['see'])


{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('b', 'o'), ('o', 'c'), ('c', 'b')},
 'walk': {('o',), ('c',)}}
True
True


In [17]:
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])
g

{'x': 'o', 'y': 'c'}

In [18]:
print(g)

g[c/y][o/x]


In [19]:
m = nltk.Model(dom, val)
print(m.evaluate('see(olive, y)', g))

True


In [22]:
print(g['y'])
print(m.evaluate('see(y, x)', g))

c
False


In [25]:
g.purge()
print(g)
print(m.evaluate('see(olive, y)', g))
print(m.evaluate('see(bertie, olive) & boy(bertie) & -walk(bertie)', g))

g
Undefined
True


In [30]:
# Quantification
print(m.evaluate('exists x.(girl(x) & walk(x))', g))
print(m.evaluate('girl(x) & walk(x)', g.add('x', 'o')))

fmla1 = lp.parse('girl(x) | boy(x)')
print(m.satisfiers(fmla1, 'x', g))
fmla2 = lp.parse('girl(x) -> walk(x)')
print(m.satisfiers(fmla2, 'x', g))
fmla3 = lp.parse('walk(x) -> girl(x)')
print(m.satisfiers(fmla3, 'x', g))

print(m.evaluate('all x.(girl(x) -> walk(x))', g))

True
True
{'o', 'b'}
{'o', 'c', 'b'}
{'o', 'b'}
True


In [33]:
v2 = """
    bruce => b
    elspeth => e
    julia => j
    matthew => m
    person => {b, e, j, m}
    admire => {(j, b), (b, b), (m, e), (e, m), (e, j)}
    """
val2 = nltk.Valuation.fromstring(v2)

dom2 = val2.domain
m2 = nltk.Model(dom2, val2)
g2 = nltk.Assignment(dom2)
fmla4 = lp.parse('(person(x) -> exists y.(person(y) & admire(x, y)))')
print(m2.satisfiers(fmla4, 'x', g2))

fmla5 = lp.parse('(person(y) & all x.(person(x) -> admire(x, y)))')
print(m2.satisfiers(fmla5, 'y', g2))

fmla6 = lp.parse('(person(y) & all x.((x = bruce | x = julia) -> admire(x, y)))')
print(m2.satisfiers(fmla6, 'y', g2))    


{'e', 'm', 'j', 'b'}
set()
{'b'}


In [35]:
a3 = lp.parse('exists x.(man(x) & walks(x))')
c1 = lp.parse('mortal(socrates)')
c2 = lp.parse('-mortal(socrates)')
# mb = nltk.Mace(5)
# need to download prover9
# print(mb.build_model(None, [a3, c1]))
# print(mb.build_model(None, [a3, c2]))
# print(mb.build_model(None, [c1, c2]))


In [38]:
a4 = lp.parse('exists y. (woman(y) & all x. (man(x) -> love(x,y)))')
a5 = lp.parse('man(adam)')
a6 = lp.parse('woman(eve)')
g = lp.parse('love(adam,eve)')
# mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6])
# print(mc.build_model())
# print(mc.valuation)
# need to download prover9

In [41]:
a7 = lp.parse('all x. (man(x) -> -woman(x))')
g = lp.parse('love(adam,eve)')
# mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6, a7])
# print(mc.build_model())
# print(mc.valuation)
# need to download prover9

In [43]:
lp = nltk.sem.logic.LogicParser()
e = lp.parse(r'\x.(walk(x) & chew_gum(x))')
print(e)
print(e.free())
print(e.simplify())
print(lp.parse(r'\x.(walk(x) & chew_gum(y))'))



\x.(walk(x) & chew_gum(x))
set()
\x.(walk(x) & chew_gum(x))
\x.(walk(x) & chew_gum(y))


In [44]:
e = lp.parse(r'\x.(walk(x) & chew_gum(x))(gerald)')
print(e)
print(e.simplify())

print(lp.parse(r'\x.\y.(dog(x) & own(y, x))(cyril)').simplify())
print(lp.parse(r'\x y.(dog(x) & own(y, x))(cyril, angus)').simplify())



\x.(walk(x) & chew_gum(x))(gerald)
(walk(gerald) & chew_gum(gerald))


In [46]:
e1 = lp.parse('exists x.P(x)')
print(e1)
e2 = e1.alpha_convert(nltk.Variable('z'))
print(e2)
print(e1 == e2)

e3 = lp.parse('exists x.P(x)')
print(e3)
print(e3.simplify())

exists x.P(x)
exists z.P(z)
True
exists x.P(x)
exists x.P(x)


In [47]:
lp = nltk.sem.logic.LogicParser()
tvp = lp.parse(r'\X x.X(\y.chase(x,y))')
np = lp.parse(r'(\P.exists x.(dog(x) & P(x)))')
vp = nltk.sem.ApplicationExpression(tvp, np)
print(vp)
print(vp.simplify())

(\X x.X(\y.chase(x,y)))(\P.exists x.(dog(x) & P(x)))
\x.exists z1.(dog(z1) & chase(x,z1))


In [69]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)
# grammar = nltk.data.load('grammars/book_grammars/simple-sem.fcfg')  # Replace 'atis.cfg' with the grammar you want to use
# parser = nltk.ChartParser(grammar)

sentence = 'Angus gives a bone to every dog'

tokens = sentence.split()
# trees = parser.nbest_parse(tokens)
trees = parser.parse(tokens)
print(next(trees))
for tree in trees:
    print(tree.label()['SEM'])




(S[SEM=<all z9.(dog(z9) -> exists z8.(bone(z8) & give(angus,z8,z9)))>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(angus)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(angus)>] Angus))
  (VP[NUM='sg', SEM=<\x.all z9.(dog(z9) -> exists z8.(bone(z8) & give(x,z8,z9)))>]
    (DTV[NUM='sg', SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>, TNS='pres']
      gives)
    (NP[NUM='sg', SEM=<\Q.exists x.(bone(x) & Q(x))>]
      (Det[NUM='sg', SEM=<\P Q.exists x.(P(x) & Q(x))>] a)
      (Nom[NUM='sg', SEM=<\x.bone(x)>]
        (N[NUM='sg', SEM=<\x.bone(x)>] bone)))
    (PP[SEM=<\Q.all x.(dog(x) -> Q(x))>, +TO]
      (P[+to] to)
      (NP[NUM='sg', SEM=<\Q.all x.(dog(x) -> Q(x))>]
        (Det[NUM='sg', SEM=<\P Q.all x.(P(x) -> Q(x))>] every)
        (Nom[NUM='sg', SEM=<\x.dog(x)>]
          (N[NUM='sg', SEM=<\x.dog(x)>] dog))))))


In [89]:
v = """
bertie => b
olive => o
cyril => c
boy => {b}
girl => {o}
dog => {c}
walk => {o, c}
see => {(b, o), (c, b), (o, c)}
"""

val = nltk.Valuation.fromstring(v)

g = nltk.Assignment(val.domain)
m = nltk.Model(val.domain, val)
 
sent = 'Cyril sees every boy'
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
results = nltk.evaluate_sents([sent], grammar_file, m, g)[0]
for (syntree, semrep, value) in results:
    print(semrep)
print(value)

all z26.(boy(z26) -> see(cyril,z26))
True


In [91]:
from nltk.sem import cooper_storage as cs
sentence = 'every girl chases a dog'
trees = cs.parse_with_bindops(sentence, grammar='grammars/book_grammars/storage.fcfg')
semrep = trees[0].label()['SEM']
cs_semrep = cs.CooperStore(semrep)
print(cs_semrep.core)
for bo in cs_semrep.store:
    print(bo)

print(cs_semrep.s_retrieve(trace=True))
for reading in cs_semrep.readings:
    print(reading)

chase(z2,z3)
bo(\P.all x.(girl(x) -> P(x)),z2)
bo(\P.exists x.(dog(x) & P(x)),z3)
Permutation 1
   (\P.all x.(girl(x) -> P(x)))(\z2.chase(z2,z3))
   (\P.exists x.(dog(x) & P(x)))(\z3.all x.(girl(x) -> chase(x,z3)))
Permutation 2
   (\P.exists x.(dog(x) & P(x)))(\z3.chase(z2,z3))
   (\P.all x.(girl(x) -> P(x)))(\z2.exists x.(dog(x) & chase(z2,x)))
None
exists x.(dog(x) & all z31.(girl(z31) -> chase(z31,x)))
all x.(girl(x) -> exists z32.(dog(z32) & chase(x,z32)))


In [101]:
import nltk
from nltk.sem.drt import DrtParser

dp = DrtParser()
drs1 = dp.parse('([x, y], [angus(x), dog(y), own(x, y)])')
print(drs1)

# dp = nltk.DrtParser()
# drs1 = dp.parse('([x, y], [angus(x), dog(y), own(x, y)])')
# print(drs1)
# print(drs1.draw())
# drs1.draw()

print(drs1.fol())
drs2 = dp.parse('([x], [walk(x)]) + ([y], [run(y)])')
print(drs2)
print(drs2.simplify())

drs3 = dp.parse('([], [(([x], [dog(x)]) -> ([y],[ankle(y), bite(x, y)]))])')
print(drs3.fol())

drs4 = dp.parse('([x, y], [angus(x), dog(y), own(x, y)])')
drs5 = dp.parse('([u, z], [PRO(u), irene(z), bite(u, z)])')
drs6 = drs4 + drs5
print(drs6)
print(drs6.simplify())
print(drs6.simplify().resolve_anaphora())


([x,y],[angus(x), dog(y), own(x,y)])
exists x y.(angus(x) & dog(y) & own(x,y))
(([x],[walk(x)]) + ([y],[run(y)]))
([x,y],[walk(x), run(y)])
all x.(dog(x) -> exists y.(ankle(y) & bite(x,y)))
(([x,y],[angus(x), dog(y), own(x,y)]) + ([u,z],[PRO(u), irene(z), bite(u,z)]))
([u,x,y,z],[angus(x), dog(y), own(x,y), PRO(u), irene(z), bite(u,z)])
([u,x,y,z],[angus(x), dog(y), own(x,y), (u = [x,y,z]), irene(z), bite(u,z)])


In [119]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/drt.fcfg', logic_parser=nltk.sem.drt.DrtParser())
trees = parser.parse('Angus owns a dog'.split())
print(next(trees))
# for tree in trees:
#     print(tree.label()['SEM'])
# trees = parser.nbest_parse('Angus owns a dog'.split())
# print(trees[0].node['sem'].simplify())

(S[SEM=<([x,z64],[Angus(x), dog(z64), own(x,z64)])>]
  (NP[-LOC, NUM='sg', SEM=<\P.(([x],[Angus(x)]) + P(x))>]
    (PropN[-LOC, NUM='sg', SEM=<\P.(([x],[Angus(x)]) + P(x))>] Angus))
  (VP[NUM='sg', SEM=<\z63.([x],[dog(x), own(z63,x)])>]
    (TV[NUM='sg', SEM=<\X x.X(\y.([],[own(x,y)]))>, tns='pres'] owns)
    (NP[NUM='sg', SEM=<\Q.(([x],[dog(x)]) + Q(x))>]
      (Det[NUM='sg', SEM=<\P Q.(([x],[]) + P(x) + Q(x))>] a)
      (Nom[NUM='sg', SEM=<\x.([],[dog(x)])>]
        (N[NUM='sg', SEM=<\x.([],[dog(x)])>] dog)))))


In [122]:
dt = nltk.DiscourseTester(['A student dances', 'Every student is a person'])
# dt.readings()
# need to download prover9
# print(dt.add_sentence('No person dances'))
# print(dt.retract_sentence('No person dances'))
# print(dt.add_sentence('A person dances', consistchk=True))




LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration parameters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <https://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [124]:
from nltk.tag import RegexpTagger
tagger = RegexpTagger(
    [('^(chases|runs)$', 'VB'),
        ('^(a)$', 'ex_quant'),
        ('^(every)$', 'univ_quant'),
        ('^(dog|boy)$', 'NN'),
        ('^(He)$', 'PRP')
    ])
# rc = nltk.DrtGlueReadingCommand(depparser=nltk.MaltParser(tagger=tagger))
# dt = nltk.DiscourseTester(['Every dog chases a boy', 'He runs'], rc)
# print(dt.readings())
# print(dt.readings(show_thread_readings=True))
# print(dt.readings(show_thread_readings=True, filter=True))
# need to download prover9